In [1]:
from dypac import dypac
import numpy as np
import sklearn as sk
import scipy as sp
import matplotlib.pyplot as plt
import time
from nilearn import datasets
from nilearn import image
from nilearn.input_data import NiftiMasker
from nilearn import plotting
from nilearn.decomposition import DictLearning, CanICA
from scipy.stats import pearsonr

ImportError: attempted relative import with no known parent package

In [ ]:
adhd_dataset = datasets.fetch_adhd(n_subjects=1)
epi_filename = adhd_dataset.func[0]

In [ ]:
model = dypac(n_clusters=10, n_states=2, verbose=1, n_init=1, n_init_aggregation=1, n_replications=20, 
              detrend=True, smoothing_fwhm=5, standardize=True, n_batch=1)
stab_maps = model.fit(epi_filename)

In [ ]:
num_comp = 7
comp = model.masker_.inverse_transform(model.components_[num_comp,:].todense())
plotting.view_img(comp, threshold=0.1, vmax=1, title="Dwell time: {dt}".format(dt=model.dwell_time_[num_comp]))

In [ ]:
tseries = model.masker_.transform(epi_filename)

In [ ]:
weights = model.transform_sparse(epi_filename)

In [ ]:
plt.plot(weights[:, num_comp])

In [ ]:
num_t = 50
img = model.masker_.inverse_transform(tseries[num_t,:])
plotting.view_img(img, cut_coords=[2, -22, 10], vmax=4)

In [ ]:
img_r = model.masker_.inverse_transform(weights[num_t,:] * model.components_)
plotting.view_img(img_r, cut_coords=[2, -22, 10], vmax=4)

In [ ]:
img_diff = image.new_img_like(img, img.get_fdata() - img_r.get_fdata())
plotting.view_img(img_diff, cut_coords=[2, -22, 10], vmax=4)

Let's calculate the original std of the data:

In [ ]:
for tt in range(0,tseries.shape[0]):
    img = model.masker_.inverse_transform(tseries[tt, :])
    if (tt==0):
        diff = np.square(img.get_fdata()) # The data is demeaned, so this is a square diff to the mean
    else:
        diff = diff + np.square(img.get_fdata())
img_orig_std = image.new_img_like(img, diff / tseries.shape[0])
plotting.view_img(img_orig_std, cut_coords=[2, -22, 10])

Let's calculate a std image of the residuals:

In [ ]:
for tt in range(0,tseries.shape[0]):
    img = model.masker_.inverse_transform(tseries[tt, :])
    img_r = model.masker_.inverse_transform(weights[tt,:] * model.components_)
    if (tt==0):
        diff = np.square(img.get_fdata() - img_r.get_fdata())
    else:
        diff = diff + np.square(img.get_fdata() - img_r.get_fdata())
img_std = image.new_img_like(img, diff / tseries.shape[0])
plotting.view_img(img_std, cut_coords=[2, -22, 10], vmax=1)